In [88]:
pip install emoji

In [89]:
import sys

import re
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
import emoji

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Data preparation

### Cleaning and tokenization

In [90]:
corpus = 'Who ❤️ "word embeddings" in 2020? I do!!!'

In [91]:
print(f'Corpus:  {corpus}')
data = re.sub(r'[,!?;-]', '.', corpus)
print(f"After clearning punctuation : ", {data})

Corpus:  Who ❤️ "word embeddings" in 2020? I do!!!
After clearning punctuation :  {'Who ❤️ "word embeddings" in 2020. I do...'}


In [92]:
print(f"Initial string : {data}")
data = nltk.word_tokenize(data)
print(f"After tokenization: {data}")

Initial string : Who ❤️ "word embeddings" in 2020. I do...
After tokenization: ['Who', '❤️', '``', 'word', 'embeddings', "''", 'in', '2020', '.', 'I', 'do', '...']


In [93]:
print(f'Initial list of tokens: {data}')
data = [ch.lower() for ch in data
        if ch.isalpha()
        or ch == "."
        or bool(emoji.emoji_list(ch))
      ]
print(f"After cleaning : {data}")

Initial list of tokens: ['Who', '❤️', '``', 'word', 'embeddings', "''", 'in', '2020', '.', 'I', 'do', '...']
After cleaning : ['who', '❤️', 'word', 'embeddings', 'in', '.', 'i', 'do']


In [94]:
def tokenize(corpus):
  data = re.sub(r'[,!?;-]+', '.', corpus)
  data = nltk.word_tokenize(data)
  data = [ch.lower() for ch in data
          if ch.isalpha()
          or ch == '.'
          or bool(emoji.emoji_list(ch))
          ]
  return data

In [95]:
corpus = 'I am happy because I am learning'
print(f'Corpus:  {corpus}')
words = tokenize(corpus)
print(f'Words (tokens):  {words}')

Corpus:  I am happy because I am learning
Words (tokens):  ['i', 'am', 'happy', 'because', 'i', 'am', 'learning']


In [96]:
tokenize("Hi John, my name is Peter. How are you?")

['hi', 'john', '.', 'my', 'name', 'is', 'peter', '.', 'how', 'are', 'you', '.']

# Sliding window of words

In [97]:
def get_windows(words, C):
  i = C
  while i < len(words) - C:
    center_word = words[i]
    context_words = words[(i - C) : i] + words[(i + 1) : (i + C + 1)]
    yield context_words, center_word
    i += 1

In [98]:
for x,y in get_windows(['i', 'am', 'happy', 'because', 'i', 'am', 'learning'], 2):
  print(f"{x}\t{y}")

['i', 'am', 'because', 'i']	happy
['am', 'happy', 'i', 'am']	because
['happy', 'because', 'am', 'learning']	i


# Transforming words into vectors for the training set

### Mapping words to indices and indices to words

In [99]:
def get_dict(words):
  words = sorted(list(set(words)))
  idx = 0
  word2Ind = {}
  Ind2word = {}
  for word in words:
    word2Ind[word] = idx
    Ind2word[idx] = word
    idx += 1
  return word2Ind, Ind2word

In [100]:
word2Ind, Ind2word = get_dict(words)

In [101]:
word2Ind

{'am': 0, 'because': 1, 'happy': 2, 'i': 3, 'learning': 4}

In [102]:
print("Index of the word 'i':  ",word2Ind['i'])

Index of the word 'i':   3


In [103]:
V = len(word2Ind)
print("Size of vocabulary: ", V)

Size of vocabulary:  5


# Getting one-hot word vectors

In [104]:
n = word2Ind['happy']
n

2

In [105]:
center_word_vector = np.zeros(V)
center_word_vector

array([0., 0., 0., 0., 0.])

In [106]:
len(center_word_vector) == V

True

In [107]:
center_word_vector[n] = 1

In [108]:
center_word_vector

array([0., 0., 1., 0., 0.])

In [109]:
def word_to_one_hot_vector(word, word2Ind, V):
  one_hot_vector = np.zeros(V)
  one_hot_vector[word2Ind[word]] = 1
  return one_hot_vector

In [110]:
word_to_one_hot_vector('happy', word2Ind, V)

array([0., 0., 1., 0., 0.])

In [111]:
word_to_one_hot_vector('learning', word2Ind, V)

array([0., 0., 0., 0., 1.])

# Getting context word vectors

In [112]:
context_words = ['i', 'am', 'because', 'i']

In [113]:
context_word_vectors = [word_to_one_hot_vector(w, word2Ind, V) for w in context_words]
context_word_vectors

[array([0., 0., 0., 1., 0.]),
 array([1., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0.]),
 array([0., 0., 0., 1., 0.])]

In [114]:
np.mean(context_word_vectors, axis = 0)

array([0.25, 0.25, 0.  , 0.5 , 0.  ])

In [115]:
def context_words_to_vector(context_words, word2Ind, V):
  context_words_vectors = [word_to_one_hot_vector(w, word2Ind, V) for w in context_words]
  context_words_vectors = np.mean(context_word_vectors, axis = 0)
  return context_words_vectors

In [116]:
context_words_to_vector(['i', 'am', 'because', 'i'], word2Ind, V)

array([0.25, 0.25, 0.  , 0.5 , 0.  ])

In [117]:
context_words_to_vector(['am', 'happy', 'i', 'am'], word2Ind, V)

array([0.25, 0.25, 0.  , 0.5 , 0.  ])

# Building the training set

In [118]:
words

['i', 'am', 'happy', 'because', 'i', 'am', 'learning']

In [119]:
for context_words, center_word in get_windows(words, 2):
  print(f"Context words: {context_words} -> {context_words_to_vector(context_words, word2Ind, V)}")
  print(f"Center words : {center_word}   -> {word_to_one_hot_vector(center_word, word2Ind, V)}")
  print()

Context words: ['i', 'am', 'because', 'i'] -> [0.25 0.25 0.   0.5  0.  ]
Center words : happy   -> [0. 0. 1. 0. 0.]

Context words: ['am', 'happy', 'i', 'am'] -> [0.25 0.25 0.   0.5  0.  ]
Center words : because   -> [0. 1. 0. 0. 0.]

Context words: ['happy', 'because', 'am', 'learning'] -> [0.25 0.25 0.   0.5  0.  ]
Center words : i   -> [0. 0. 0. 1. 0.]



In [120]:
def get_training_example(words, C, word2Ind, V):
  for context_words, center_word in get_windows(words, C):
    yield context_words_to_vector(context_words, word2Ind, V), word_to_one_hot_vector(center_word, word2Ind, V)

In [121]:
for context_words_vector, center_word_vector in get_training_example(words, 2, word2Ind, V):
  print(f"Context words vector: {context_words_vector}")
  print(f"Center word vector  : {center_word_vector}")
  print()

Context words vector: [0.25 0.25 0.   0.5  0.  ]
Center word vector  : [0. 0. 1. 0. 0.]

Context words vector: [0.25 0.25 0.   0.5  0.  ]
Center word vector  : [0. 1. 0. 0. 0.]

Context words vector: [0.25 0.25 0.   0.5  0.  ]
Center word vector  : [0. 0. 0. 1. 0.]



# Continuous bag-of-words model

### ReLU activation function

In [122]:
np.random.seed(10)
z_1 = 10 * np.random.rand(5, 1) - 5
z_1

array([[ 2.71320643],
       [-4.79248051],
       [ 1.33648235],
       [ 2.48803883],
       [-0.01492988]])

In [123]:
h = z_1.copy()

In [124]:
h < 0

array([[False],
       [ True],
       [False],
       [False],
       [ True]])

In [125]:
h[h < 0] = 0

In [126]:
h

array([[2.71320643],
       [0.        ],
       [1.33648235],
       [2.48803883],
       [0.        ]])

In [127]:
def relu(z):
  result = z.copy()
  result[result < 0] = 0
  return result

In [128]:
z = np.array([[-1.25459881], [ 4.50714306], [ 2.31993942], [ 0.98658484], [-3.4398136 ]])
relu(z)

array([[0.        ],
       [4.50714306],
       [2.31993942],
       [0.98658484],
       [0.        ]])

### Softmax activation function

In [129]:
z = np.array([9, 8, 11, 10, 8.5])
z

array([ 9. ,  8. , 11. , 10. ,  8.5])

In [130]:
e_z = np.exp(z)
e_z

array([ 8103.08392758,  2980.95798704, 59874.1417152 , 22026.46579481,
        4914.7688403 ])

In [131]:
sum_e_z = np.sum(e_z)
sum_e_z

97899.41826492078

In [132]:
e_z[0] / sum_e_z

0.08276947985173956

In [133]:
def softmax(z):
  e_z = np.exp(z)
  sum_e_z = np.sum(e_z)
  return e_z / sum_e_z

In [134]:
softmax([9, 8, 11, 10, 8.5])

array([0.08276948, 0.03044919, 0.61158833, 0.22499077, 0.05020223])

### Dimensions: 1-D arrays vs 2-D column vectors

In [135]:
x_array = np.zeros(V)
x_array

array([0., 0., 0., 0., 0.])

In [136]:
x_array.shape

(5,)

In [137]:
x_column_vector = x_array.copy()
x_column_vector.shape = (V,1)
x_column_vector

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [138]:
x_column_vector.shape

(5, 1)

### Forward propagation

In [139]:
N = 3

In [140]:
W1 = np.array([[ 0.41687358,  0.08854191, -0.23495225,  0.28320538,  0.41800106],
               [ 0.32735501,  0.22795148, -0.23951958,  0.4117634 , -0.23924344],
               [ 0.26637602, -0.23846886, -0.37770863, -0.11399446,  0.34008124]])

W2 = np.array([[-0.22182064, -0.43008631,  0.13310965],
               [ 0.08476603,  0.08123194,  0.1772054 ],
               [ 0.1871551 , -0.06107263, -0.1790735 ],
               [ 0.07055222, -0.02015138,  0.36107434],
               [ 0.33480474, -0.39423389, -0.43959196]])

b1 = np.array([[ 0.09688219],
               [ 0.29239497],
               [-0.27364426]])

b2 = np.array([[ 0.0352008 ],
               [-0.36393384],
               [-0.12775555],
               [-0.34802326],
               [-0.07017815]])

In [141]:
print(f'V (vocabulary size): {V}')
print(f'N (embedding size / size of the hidden layer): {N}')
print(f'size of W1: {W1.shape} (NxV)')
print(f'size of b1: {b1.shape} (Nx1)')
print(f'size of W2: {W2.shape} (VxN)')
print(f'size of b2: {b2.shape} (Vx1)')

V (vocabulary size): 5
N (embedding size / size of the hidden layer): 3
size of W1: (3, 5) (NxV)
size of b1: (3, 1) (Nx1)
size of W2: (5, 3) (VxN)
size of b2: (5, 1) (Vx1)


In [142]:
training_examples = get_training_example(words, 2, word2Ind, V)

In [143]:
x_array, y_array = next(training_examples)

In [144]:
x_array

array([0.25, 0.25, 0.  , 0.5 , 0.  ])

In [145]:
y_array

array([0., 0., 1., 0., 0.])

In [146]:
x = x_array.copy()
x.shape = (V,1)
print('x')
print(x)

print()

y = y_array.copy()
y.shape = (V,1)
print('y')
print(y)

x
[[0.25]
 [0.25]
 [0.  ]
 [0.5 ]
 [0.  ]]

y
[[0.]
 [0.]
 [1.]
 [0.]
 [0.]]


### Values of the hidden layer

In [147]:
z1 = np.dot(W1, x) + b1

In [148]:
z1

array([[ 0.36483875],
       [ 0.63710329],
       [-0.3236647 ]])

In [149]:
h = relu(z1)

### Value of the output layer

In [150]:
z2 = np.dot(W2, h) + b2
z2

array([[-0.31973737],
       [-0.28125477],
       [-0.09838369],
       [-0.33512159],
       [-0.19919612]])

In [151]:
y_hat = softmax(z2)
y_hat

array([[0.18519074],
       [0.19245626],
       [0.23107446],
       [0.18236353],
       [0.20891502]])

### Cross-entropy loss

In [152]:
y_hat

array([[0.18519074],
       [0.19245626],
       [0.23107446],
       [0.18236353],
       [0.20891502]])

In [153]:
y

array([[0.],
       [0.],
       [1.],
       [0.],
       [0.]])

In [154]:
def cross_entropy_loss(y_predicted, y_actual):
  loss = np.sum(-np.log(y_predicted) * y_actual)
  return loss

In [155]:
cross_entropy_loss(y_hat, y)

1.4650152923611106

### Backpropagation

In [156]:
grad_b2 = y_hat - y

grad_b2

array([[ 0.18519074],
       [ 0.19245626],
       [-0.76892554],
       [ 0.18236353],
       [ 0.20891502]])

In [157]:
grad_W2 = np.dot(y_hat - y, h.T)

grad_W2

array([[ 0.06756476,  0.11798563,  0.        ],
       [ 0.0702155 ,  0.12261452,  0.        ],
       [-0.28053384, -0.48988499,  0.        ],
       [ 0.06653328,  0.1161844 ,  0.        ],
       [ 0.07622029,  0.13310045,  0.        ]])

In [158]:
grad_b1 = relu(np.dot(W2.T, y_hat - y))

grad_b1

array([[0.        ],
       [0.        ],
       [0.17045858]])

In [159]:
grad_W1 = np.dot(relu(np.dot(W2.T, y_hat - y)), x.T)

grad_W1

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.04261464, 0.04261464, 0.        , 0.08522929, 0.        ]])

In [160]:
print(f'V (vocabulary size): {V}')
print(f'N (embedding size / size of the hidden layer): {N}')
print(f'size of grad_W1: {grad_W1.shape} (NxV)')
print(f'size of grad_b1: {grad_b1.shape} (Nx1)')
print(f'size of grad_W2: {grad_W2.shape} (VxN)')
print(f'size of grad_b2: {grad_b2.shape} (Vx1)')

V (vocabulary size): 5
N (embedding size / size of the hidden layer): 3
size of grad_W1: (3, 5) (NxV)
size of grad_b1: (3, 1) (Nx1)
size of grad_W2: (5, 3) (VxN)
size of grad_b2: (5, 1) (Vx1)


### Gradient descent

In [161]:
# learning rate
alpha = 0.03

In [162]:
W1_new = W1 - alpha * grad_W1

In [163]:
print('old value of W1:')
print(W1)
print()
print('new value of W1:')
print(W1_new)

old value of W1:
[[ 0.41687358  0.08854191 -0.23495225  0.28320538  0.41800106]
 [ 0.32735501  0.22795148 -0.23951958  0.4117634  -0.23924344]
 [ 0.26637602 -0.23846886 -0.37770863 -0.11399446  0.34008124]]

new value of W1:
[[ 0.41687358  0.08854191 -0.23495225  0.28320538  0.41800106]
 [ 0.32735501  0.22795148 -0.23951958  0.4117634  -0.23924344]
 [ 0.26509758 -0.2397473  -0.37770863 -0.11655134  0.34008124]]


In [164]:
W2_new = W2 - alpha * grad_W2
b1_new = b1 - alpha * grad_b1
b2_new = b2 - alpha * grad_b2

print('W2_new')
print(W2_new)
print()
print('b1_new')
print(b1_new)
print()
print('b2_new')
print(b2_new)

W2_new
[[-0.22384758 -0.43362588  0.13310965]
 [ 0.08265956  0.0775535   0.1772054 ]
 [ 0.19557112 -0.04637608 -0.1790735 ]
 [ 0.06855622 -0.02363691  0.36107434]
 [ 0.33251813 -0.3982269  -0.43959196]]

b1_new
[[ 0.09688219]
 [ 0.29239497]
 [-0.27875802]]

b2_new
[[ 0.02964508]
 [-0.36970753]
 [-0.10468778]
 [-0.35349417]
 [-0.0764456 ]]


# Extracting word embedding vectors

### Option 1 : extract embedding vectors from W1

In [165]:
W1

array([[ 0.41687358,  0.08854191, -0.23495225,  0.28320538,  0.41800106],
       [ 0.32735501,  0.22795148, -0.23951958,  0.4117634 , -0.23924344],
       [ 0.26637602, -0.23846886, -0.37770863, -0.11399446,  0.34008124]])

In [166]:
for i in range(V):
    print(Ind2word[i])

am
because
happy
i
learning


In [167]:
# loop through each word of the vocabulary
for word in word2Ind:
    # extract the column corresponding to the index of the word in the vocabulary
    word_embedding_vector = W1[:, word2Ind[word]]

    print(f'{word}: {word_embedding_vector}')

am: [0.41687358 0.32735501 0.26637602]
because: [ 0.08854191  0.22795148 -0.23846886]
happy: [-0.23495225 -0.23951958 -0.37770863]
i: [ 0.28320538  0.4117634  -0.11399446]
learning: [ 0.41800106 -0.23924344  0.34008124]


### Option 2 : extract embedding vectors from W2

In [168]:
W2.T

array([[-0.22182064,  0.08476603,  0.1871551 ,  0.07055222,  0.33480474],
       [-0.43008631,  0.08123194, -0.06107263, -0.02015138, -0.39423389],
       [ 0.13310965,  0.1772054 , -0.1790735 ,  0.36107434, -0.43959196]])

In [169]:
# loop through each word of the vocabulary
for word in word2Ind:
    # extract the column corresponding to the index of the word in the vocabulary
    word_embedding_vector = W2.T[:, word2Ind[word]]

    print(f'{word}: {word_embedding_vector}')

am: [-0.22182064 -0.43008631  0.13310965]
because: [0.08476603 0.08123194 0.1772054 ]
happy: [ 0.1871551  -0.06107263 -0.1790735 ]
i: [ 0.07055222 -0.02015138  0.36107434]
learning: [ 0.33480474 -0.39423389 -0.43959196]


### Option 3: extract embedding vectors from W1 and W2

In [170]:
W3 = (W1+W2.T)/2

W3

array([[ 0.09752647,  0.08665397, -0.02389858,  0.1768788 ,  0.3764029 ],
       [-0.05136565,  0.15459171, -0.15029611,  0.19580601, -0.31673866],
       [ 0.19974284, -0.03063173, -0.27839106,  0.12353994, -0.04975536]])

In [171]:
# loop through each word of the vocabulary
for word in word2Ind:
    # extract the column corresponding to the index of the word in the vocabulary
    word_embedding_vector = W3[:, word2Ind[word]]

    print(f'{word}: {word_embedding_vector}')

am: [ 0.09752647 -0.05136565  0.19974284]
because: [ 0.08665397  0.15459171 -0.03063173]
happy: [-0.02389858 -0.15029611 -0.27839106]
i: [0.1768788  0.19580601 0.12353994]
learning: [ 0.3764029  -0.31673866 -0.04975536]
